# Table

In [1]:
import geopandas as gpd

directory = '/Users/arthurcalvi/Data/results/disturbances/matching_france_v02_EPSG3857.parquet'
gdf = gpd.read_parquet(directory)


# Research

In [29]:
def get_nd(item, daoi, geometry):
    if dplatform[ item.properties['platform'] ] == 'landsat-7':
        _, nd = get_cc(item, geometry, daoi, config)
        item.properties.setdefault('nodata', nd)
    return None

In [53]:
import json 
def is_pan_available(item, geometry):
    sat = dplatform[item.properties['platform']]
    collection = config[sat]['pan']['collection']
    try : 
        item_pan = collection.get_item( item.id.replace(config[sat]['pan']['from'], config[sat]['pan']['to']) )
        metadata = json.loads(item_pan.assets.download_as_str('MTL_JSON'))
        pan = item_pan.assets.crop_as_array(config[sat]['pan']['pan_band'], bbox= shape(geometry).bounds)
        return True
    except : 
        return False
    
def get_res(item, geometry):
    sat = dplatform[item.properties['platform']]
    band = config[sat]['rgb_bands'][0]
    res = config[sat]['resolution'][band]
    if 'pan' in config[sat].keys() and is_pan_available(item, geometry):
        res = config[sat]['resolution']['pan']
    item.properties['resolution'] = res

In [54]:
%load_ext autoreload
%autoreload 2

from chronos import research_items, get_sats, priority, config, dplatform, check_intervals, get_cc
import pandas as pd
import numpy as np 
from shapely.geometry import shape
from rasterio.features import rasterize
from rasterio.warp import transform_geom
from copy import deepcopy
from datetime import datetime, timezone
from joblib import Parallel, delayed

verbose=1
index = 10
buffer = 3000 #meters
delta_min = 20
cc1 = 1
cc2 = 75
start_date = datetime(2013, 1, 1, 10, 10, 10, tzinfo=timezone.utc)
end_date = datetime(2014, 12, 31, 10, 10, 10, tzinfo=timezone.utc)
geometry = gdf.iloc[index]['geometry']

#geometry
old_geometry = deepcopy(geometry)
geometry = transform_geom('epsg:3857', 'epsg:4326', shape(old_geometry).convex_hull)
geometry_buffer = transform_geom('epsg:3857', 'epsg:4326', shape(old_geometry).convex_hull.buffer(buffer))

#sats 
count_sat = {}
sats = get_sats(start_date, end_date, config, priority)
if verbose > 0:
    print(sats)
for sat in sats:
    count_sat[sat] = 0

#RESEARCH 1
items1 = research_items(geometry, start_date, end_date, sats, config, cc1)

#retrieve crs, transform and array for each sat

items1_sat = set([dplatform[item.properties['platform']] for item in items1])
items1_prop = {sat:None for sat in items1_sat}
for sat in items1_sat:
    i = 0 
    temp_items1 = [item for item in items1 if dplatform[item.properties['platform']] == sat]
    while items1_prop[sat] is None and i < len(temp_items1):
        item = temp_items1[i]
        try :
            crs, transfo, arr, _ = item.assets.crop_as_array(config[ dplatform[ item.properties['platform'] ] ]['qa'][0], bbox= shape(geometry).bounds)
            items1_prop[sat] = {'crs':crs, 'transfo':transfo, 'arr':arr}
        except :
            pass
        i += 1

items1_sat = [sat for sat in items1_sat if items1_prop[sat] is not None]
#retrieve aoi for each sat
daoi = {}
for sat in items1_sat:
    daoi[sat] = rasterize([transform_geom('epsg:3857', items1_prop[sat]['crs'] , old_geometry.buffer(100).convex_hull)], out_shape = items1_prop[sat]['arr'].shape[1:], transform=items1_prop[sat]['transfo'], fill=np.nan, all_touched=False)

#research and filter for landsat-7
_ = Parallel(n_jobs=-1, prefer='threads', verbose=verbose)(delayed(get_nd)(item, daoi, geometry) for item in items1)
items1 = [item for item in items1 if ('nodata' not in item.properties or item.properties['nodata'] <= 25)]

if verbose > 0:
            print(f'Items found for research 1 with cc={cc1} : {len(items1)}')
x = [start_date] + [item.properties['datetime'] for item in items1] + [end_date]
y = [None] + [sats.index( dplatform[ item.properties['platform'] ]) for item in items1] + [None]

#gap
df = pd.DataFrame(data=np.array([x,y]).T, columns=['date', 'sat'])
df['gap'] = (df.date - df.date.shift()).dt.days
df['hole'] = (df.gap > delta_min)
dates_hole = [[df.date.iloc[x-1], df.date.iloc[x]] for x in df.index.to_numpy()[df.hole]]

#RESEARCH 2 
items2 = research_items(geometry, start_date, end_date, sats, config, cc2)
dates = [item.properties['datetime'] for item in items2]
indexes_prob = [check_intervals(date, dates_hole, 1) for date in dates]
items_prob = [items2[i] for i in range(len(items2)) if indexes_prob[i]]


#retrieve crs, transform and array for each sat 
items2_sat = set([dplatform[item.properties['platform']] for item in items2])
items2_prop = items1_prop
if not sorted(items2_sat) == sorted(items1_sat):
    for sat in items2_sat:
        if sat not in items1_sat:
            count_sat[sat] = 0
            i = 0 
            temp_items2 = [item for item in items2 if dplatform[item.properties['platform']] == sat]
            while items2_prop[sat] is None and i < len(temp_items2):
                item = temp_items2[i]
                try :
                    crs, transfo, arr, _ = item.assets.crop_as_array(config[ dplatform[ item.properties['platform'] ] ]['qa'][0], bbox= shape(geometry).bounds)
                    items2_prop[sat] = {'crs':crs, 'transfo':transfo, 'arr':arr}
                except :
                    pass
                i += 1

items2 = [item for item in items2 if items2_prop[dplatform[item.properties['platform']]] is not None]
items2_sat = [sat for sat in items2_sat if items2_prop[sat] is not None]

if len(items2) > 0:
    daoi = {}
    for sat in items2_sat:
        daoi[sat] = rasterize([transform_geom('epsg:3857', items2_prop[sat]['crs'] , old_geometry.buffer(100).convex_hull)], out_shape = items2_prop[sat]['arr'].shape[1:], transform=items2_prop[sat]['transfo'], fill=np.nan, all_touched=False)
    cc_nodata = Parallel(n_jobs=-1, prefer='threads', verbose=verbose)(delayed(get_cc)(item, geometry, daoi, config) for item in items_prob)
    indexes_cc_ok = (np.array(cc_nodata)[:,0] < 25)
    indexes_nodata_ok = (np.array(cc_nodata)[:,1] < 25)
    cloud_cover = np.array(cc_nodata)[indexes_cc_ok, 0]
    nodata_cover = np.array(cc_nodata)[indexes_nodata_ok, 1]
    items_ok = [items_prob[i] for i in range(len(items_prob)) if (indexes_cc_ok[i] and indexes_nodata_ok[i])]

    #update CC:
    for i,item in enumerate(items_ok):
        item.properties['eo:cloud_cover'] = cloud_cover[i]
        item.properties['nodata'] = nodata_cover[i]

    if verbose > 0:
        print(f'Items found for research 2 with cc={cc1} (on aoi) : {len(items2)}')

    #assembling
    items1.extend(items_ok)
    items1 = sorted(items1, key=lambda x:x.properties['datetime'])
    _ = Parallel(n_jobs=-1, prefer='threads', verbose=verbose)(delayed(get_res)(item, geometry) for item in items1)
    if verbose > 0:
        print('Items found for combined research: ', len(items1))

    #filtering
    x = [item.properties['datetime'] for item in items1]
    cc = [item.properties['eo:cloud_cover'] for item in items1]
    nd = [item.properties.get('nodata', np.nan) for item in items1]
    res = [item.properties['resolution'] for item in items1]

    y = [sats.index(dplatform[ item.properties['platform'] ]) for item in items1]
    df = pd.DataFrame(data=np.array([x,y,cc,nd, res]).T, columns=['date', 'sat', 'cc', 'nodata', 'resolution'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['landsat-5', 'landsat-7', 'landsat-8']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.4s finished


Items found for research 1 with cc=1 : 4


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 out of  38 | elapsed:    9.4s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


Items found for research 2 with cc=1 (on aoi) : 42
Items found for combined research:  23


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:    7.7s finished


In [55]:
df

,date,sat,cc,nodata,resolution
0,2013-02-22 10:55:42.684562+00:00,1,0.0,0.0,15
1,2013-06-30 10:55:04.917500+00:00,1,0.0,0.0,15
2,2013-07-08 11:01:29.759381+00:00,2,0.02,NaN,15
3,2013-07-24 11:01:28.498029+00:00,2,3.333333,0.0,15
4,2013-08-25 11:01:33.013069+00:00,2,0.0,0.0,15
5,2013-09-02 10:55:02.799937+00:00,1,1,0.0,15
6,2013-09-26 11:01:22.235455+00:00,2,0.0,0.0,15
7,2013-11-13 11:01:13.397354+00:00,2,0.0,0.0,15
8,2013-11-21 10:55:44.416125+00:00,1,10.0,0.0,15
9,2013-12-07 10:55:53.691562+00:00,1,0.0,0.0,15


In [67]:
c_cc=1
c_nodata=1
c_res=1
df['cost'] = c_cc * df.cc / 10 + c_nodata * df.nodata / 10 + c_res * df.resolution / df.resolution.max()

In [68]:
def cost(alpha:np.array, df:pd.DataFrame, days=20):
    cost = df['cost'].iloc[:,alpha.astype(bool)]
    dates = df['date'].iloc[:,alpha.astype(bool)]
    constraint = (dates - dates.shift()).dt.days
    return cost + abs(constraint.mean() - days) + constraint.std()



,date,sat,cc,nodata,resolution,cost
0,2013-02-22 10:55:42.684562+00:00,1,0.0,0.0,15,0.5
1,2013-06-30 10:55:04.917500+00:00,1,0.0,0.0,15,0.5
2,2013-07-08 11:01:29.759381+00:00,2,0.02,NaN,15,NaN
3,2013-07-24 11:01:28.498029+00:00,2,3.333333,0.0,15,0.833333
4,2013-08-25 11:01:33.013069+00:00,2,0.0,0.0,15,0.5
5,2013-09-02 10:55:02.799937+00:00,1,1,0.0,15,0.6
6,2013-09-26 11:01:22.235455+00:00,2,0.0,0.0,15,0.5
7,2013-11-13 11:01:13.397354+00:00,2,0.0,0.0,15,0.5
8,2013-11-21 10:55:44.416125+00:00,1,10.0,0.0,15,1.5
9,2013-12-07 10:55:53.691562+00:00,1,0.0,0.0,15,0.5


In [57]:
a = items_prob[0].assets.crop_as_array(config[ dplatform[ item.properties['platform'] ] ]['rgb_bands'][0], bbox= shape(geometry).bounds)